In [1]:
import polars as pl

In [4]:
qb_data = pl.read_csv("draft_picks.csv")

In [5]:
qb_data

shape: (12_253, 10)
┌────────┬──────┬───────┬──────┬───┬───────────┬──────┬──────────┬──────────┐
│ season ┆ team ┆ round ┆ pick ┆ … ┆ player_id ┆ side ┆ category ┆ position │
│ ---    ┆ ---  ┆ ---   ┆ ---  ┆   ┆ ---       ┆ ---  ┆ ---      ┆ ---      │
│ i64    ┆ str  ┆ i64   ┆ i64  ┆   ┆ str       ┆ str  ┆ str      ┆ str      │
╞════════╪══════╪═══════╪══════╪═══╪═══════════╪══════╪══════════╪══════════╡
│ 1980   ┆ DET  ┆ 1     ┆ 1    ┆ … ┆ null      ┆ O    ┆ RB       ┆ RB       │
│ 1980   ┆ NYJ  ┆ 1     ┆ 2    ┆ … ┆ null      ┆ O    ┆ WR       ┆ WR       │
│ 1980   ┆ CIN  ┆ 1     ┆ 3    ┆ … ┆ null      ┆ O    ┆ OL       ┆ T        │
│ 1980   ┆ GB   ┆ 1     ┆ 4    ┆ … ┆ null      ┆ D    ┆ DL       ┆ DE       │
│ 1980   ┆ BAL  ┆ 1     ┆ 5    ┆ … ┆ null      ┆ O    ┆ RB       ┆ RB       │
│ …      ┆ …    ┆ …     ┆ …    ┆ … ┆ …         ┆ …    ┆ …        ┆ …        │
│ 2024   ┆ LAC  ┆ 7     ┆ 253  ┆ … ┆ null      ┆ O    ┆ WR       ┆ WR       │
│ 2024   ┆ LA   ┆ 7     ┆ 254  ┆ … ┆ null      ┆ O    ┆ OL       ┆ OL       │
│ 2024   ┆ GB   ┆ 7     ┆ 255  ┆ … ┆ null      ┆ D    ┆ DB       ┆ CB       │
│ 2024   ┆ DEN  ┆ 7     ┆ 256  ┆ … ┆ null      ┆ O    ┆ OL       ┆ OL       │
│ 2024   ┆ NYJ  ┆ 7     ┆ 257  ┆ … ┆ null      ┆ null ┆ null     ┆ null     │
└────────┴──────┴───────┴──────┴───┴───────────┴──────┴──────────┴──────────┘

In [7]:
import polars as pl

qb_data = pl.read_csv("draft_picks.csv")

# Compute position breakdown by decade
breakdown = (
    qb_data.with_columns([((pl.col("season") // 10) * 10).alias("decade_start")])
    .with_columns(
        [
            (
                pl.col("decade_start").cast(pl.Utf8)
                + "-"
                + (pl.col("decade_start") + 9).cast(pl.Utf8)
            ).alias("decade")
        ]
    )
    .group_by(["decade", "position"])
    .len()
    .with_columns(
        [
            (pl.col("len") / pl.col("len").sum().over("decade") * 100)
            .round(2)
            .alias("percentage")
        ]
    )
    .sort(["decade", "percentage"], descending=[False, True])
)

# Print formatted report
for decade in sorted(breakdown["decade"].unique()):
    print(f"\n{decade}")
    print("Positions:")
    decade_rows = breakdown.filter(pl.col("decade") == decade)
    for row in decade_rows.iter_rows(named=True):
        print(f"  {row['position']}: {row['percentage']}%")

# Save as CSV
breakdown.write_csv("position_breakdown_by_decade.csv")

# Or save as formatted text report
with open("position_report.txt", "w") as f:
    for decade in sorted(breakdown["decade"].unique()):
        f.write(f"\n{decade}\n")
        f.write("Positions:\n")
        decade_rows = breakdown.filter(pl.col("decade") == decade)
        for row in decade_rows.iter_rows(named=True):
            f.write(f"  {row['position']}: {row['percentage']}%\n")


1980-1989
Positions:
  DB: 16.8%
  LB: 15.03%
  RB: 13.56%
  WR: 11.68%
  DE: 7.37%
  T: 6.92%
  G: 6.29%
  TE: 5.63%
  QB: 4.61%
  DT: 4.01%
  C: 2.93%
  NT: 2.43%
  K: 1.44%
  P: 1.29%
  KR: 0.03%

1990-1999
Positions:
  DB: 17.7%
  LB: 13.56%
  WR: 12.37%
  RB: 11.07%
  DE: 8.68%
  G: 7.34%
  DT: 6.86%
  T: 6.74%
  TE: 6.0%
  QB: 4.66%
  C: 2.68%
  P: 0.93%
  K: 0.71%
  NT: 0.67%
  KR: 0.04%

2000-2009
Positions:
  DB: 19.36%
  WR: 13.09%
  LB: 12.39%
  RB: 9.06%
  DE: 8.74%
  T: 8.27%
  DT: 7.88%
  TE: 6.0%
  G: 5.53%
  QB: 5.1%
  C: 2.67%
  K: 0.98%
  P: 0.82%
  NT: 0.12%

2010-2019
Positions:
  LB: 12.94%
  WR: 12.31%
  DB: 10.66%
  RB: 9.16%
  DE: 9.04%
  DT: 7.9%
  T: 7.0%
  CB: 5.78%
  TE: 5.74%
  G: 5.07%
  QB: 4.6%
  S: 3.62%
  C: 2.32%
  OL: 1.57%
  P: 0.71%
  K: 0.71%
  NT: 0.43%
  DL: 0.24%
  LS: 0.2%

2020-2029
Positions:
  WR: 12.95%
  OL: 11.45%
  LB: 10.57%
  DB: 9.16%
  RB: 7.67%
  DE: 7.4%
  CB: 7.05%
  TE: 5.55%
  DL: 5.02%
  DT: 4.93%
  QB: 4.23%
  S: 3.96%
  T: 

In [1]:
import polars as pl

qb_data = pl.read_csv("draft_picks.csv")

# First get individual counts
individual = (
    qb_data.with_columns([((pl.col("season") // 10) * 10).alias("decade_start")])
    .with_columns(
        [
            (
                pl.col("decade_start").cast(pl.Utf8)
                + "-"
                + (pl.col("decade_start") + 9).cast(pl.Utf8)
            ).alias("decade")
        ]
    )
    .group_by(["decade", "position"])
    .len()
)

# Then get combined
breakdown = (
    qb_data.with_columns([((pl.col("season") // 10) * 10).alias("decade_start")])
    .with_columns(
        [
            (
                pl.col("decade_start").cast(pl.Utf8)
                + "-"
                + (pl.col("decade_start") + 9).cast(pl.Utf8)
            ).alias("decade")
        ]
    )
    .with_columns(
        [
            pl.when(pl.col("position").is_in(["DB", "WR"]))
            .then(pl.lit("DB + WR"))
            .otherwise(pl.col("position"))
            .alias("position_combined")
        ]
    )
    .group_by(["decade", "position_combined"])
    .len()
    .with_columns(
        [
            (pl.col("len") / pl.col("len").sum().over("decade") * 100)
            .round(2)
            .alias("percentage")
        ]
    )
    .sort(["decade", "percentage"], descending=[False, True])
)

# Print with details
for decade in sorted(breakdown["decade"].unique()):
    print(f"\n{decade}")
    print("Positions:")
    decade_rows = breakdown.filter(pl.col("decade") == decade)

    for row in decade_rows.iter_rows(named=True):
        if row["position_combined"] == "DB + WR":
            # Get individual counts
            db_count = individual.filter(
                (pl.col("decade") == decade) & (pl.col("position") == "DB")
            )["len"].to_list()
            wr_count = individual.filter(
                (pl.col("decade") == decade) & (pl.col("position") == "WR")
            )["len"].to_list()

            db_val = db_count[0] if db_count else 0
            wr_val = wr_count[0] if wr_count else 0

            print(f"  DB + WR: {row['percentage']}% (DB: {db_val}, WR: {wr_val})")
        else:
            print(f"  {row['position_combined']}: {row['percentage']}%")


1980-1989
Positions:
  DB + WR: 28.47% (DB: 561, WR: 390)
  LB: 15.03%
  RB: 13.56%
  DE: 7.37%
  T: 6.92%
  G: 6.29%
  TE: 5.63%
  QB: 4.61%
  DT: 4.01%
  C: 2.93%
  NT: 2.43%
  K: 1.44%
  P: 1.29%
  KR: 0.03%

1990-1999
Positions:
  DB + WR: 30.07% (DB: 475, WR: 332)
  LB: 13.56%
  RB: 11.07%
  DE: 8.68%
  G: 7.34%
  DT: 6.86%
  T: 6.74%
  TE: 6.0%
  QB: 4.66%
  C: 2.68%
  P: 0.93%
  K: 0.71%
  NT: 0.67%
  KR: 0.04%

2000-2009
Positions:
  DB + WR: 32.46% (DB: 494, WR: 334)
  LB: 12.39%
  RB: 9.06%
  DE: 8.74%
  T: 8.27%
  DT: 7.88%
  TE: 6.0%
  G: 5.53%
  QB: 5.1%
  C: 2.67%
  K: 0.98%
  P: 0.82%
  NT: 0.12%

2010-2019
Positions:
  DB + WR: 22.97% (DB: 271, WR: 313)
  LB: 12.94%
  RB: 9.16%
  DE: 9.04%
  DT: 7.9%
  T: 7.0%
  CB: 5.78%
  TE: 5.74%
  G: 5.07%
  QB: 4.6%
  S: 3.62%
  C: 2.32%
  OL: 1.57%
  K: 0.71%
  P: 0.71%
  NT: 0.43%
  DL: 0.24%
  LS: 0.2%

2020-2029
Positions:
  DB + WR: 22.11% (DB: 104, WR: 147)
  OL: 11.45%
  LB: 10.57%
  RB: 7.67%
  DE: 7.4%
  CB: 7.05%
  TE: 